In [5]:
import numpy as np
import pandas as pd
import torch as th
import matplotlib.pyplot as plt
import gym as gm


минимальная реализация REINFORCE алгоритма:

реализация класса генерации стратегии PI

In [2]:
gamma = 0.99
class PI(th.nn.Module):

    def __init__(self, in_dim, out_dim, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.layers = [
            th.nn.Linear(in_dim, 64),
            th.nn.ReLU(),
            th.nn.Linear(64, out_dim)
        ]
        self.model = th.nn.Sequential(*self.layers)
        self.onpolicy_reset()
        self.train()
    
    def onpolicy_reset(self):

        self.log_probs_list = []
        self.rewards_list = []
    
    def forward(self, x):
        
        self.pd_param = self.model(x)
    
    def act(self, state):

        self.x = th.from_numpy(state.astype(np.float32))
        self.forward(self.x)
        self.pd = th.distributions.Categorical(self.pd_param)
        self.action = self.pd.sample()
        self.log_prob = self.pd.log_prob(self.action)
        self.log_probs_list.append(self.log_prob)
        
        return self.action.item()



реализация цикла обучения train

In [3]:
def train(pi, optimizer):

    T = len(pi.rewards_list)
    rets = np.empty(T, dtype=np.float32)
    future_ret = 0.0

    for curent_t in range(T):
        future_ret = pi.rewards_list[curent_t] + gamma * future_ret
        rets[curent_t] = future_ret
    
    rets = th.tensor(rets)
    log_probs_tensor = th.stack(pi.log_probs_list)
    loss = -log_probs_tensor * rets
    loss = th.sum(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss


основной цикл обучения main

In [4]:
def main():

    env = gm.make("CartPole-v0")
    in_dim = env.observation_space.shape[0]
    out_dim = env.action_space.n
    pi = PI(in_dim, out_dim)
    optimizer = th.optim.Adam(pi.parameters(), lr=0.01)
    for epi in range(300):

        state = env.reset()[0]
        for t in range(200):
            print(state)
            action = pi.act(state)
            state, reward, done, truncated, info = env.step(action)
            pi.rewards_list.append(reward)
            env.render()
            if done:
                break
            
        loss = train(pi, optimizer)
        total_reward = sum(pi.rewards_list)
        solved = total_reward > 195.0
        pi.onpolicy_reset()
        print(f"Epizode {epi}, loss: {loss}, total_reward: {total_reward}, solved: {solved}")
        print(pi.rewards_list)

main()

c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(


[-0.01055631  0.03225228  0.03224316 -0.03608451]
[-0.00991127  0.22689737  0.03152147 -0.31842256]
[-0.00537332  0.03134099  0.02515302 -0.01596791]
[-0.0047465   0.22609335  0.02483366 -0.30060986]
[-2.2463109e-04  4.2085272e-01  1.8821467e-02 -5.8535844e-01]
[ 0.00819242  0.615706    0.0071143  -0.8720535 ]
[ 0.02050654  0.8107305  -0.01032677 -1.1624912 ]
[ 0.03672116  1.0059854  -0.0335766  -1.4583939 ]
[ 0.05684086  1.2015028  -0.06274448 -1.7613744 ]


ValueError: Expected parameter probs (Tensor of shape (2,)) of distribution Categorical(probs: torch.Size([2])) to satisfy the constraint Simplex(), but found invalid values:
tensor([-0.0526,  1.0526], grad_fn=<DivBackward0>)